In [ ]:
# !pip install pandas

In [ ]:
# !pip install Wheel

In [ ]:
# !pip install scikit-learn

# 실습 1. Penguins

- 분석 목표 : 펭귄의 종(species) 예측

- 독립변수(원인): 펭귄의 생김새(연속형), 서식 지역(범주형), 성별(범주형)

- 종속변수(결과): 펭귄의 종

- 모형 : 앙상블(ensemble) 기법 사용 , 로지스틱 회귀분석도 사용 가능

[데이터 분석의 과정]

> 여기 과정이 가장 중요합니당...

- 데이터 획득
- 데이터 전처리 : 결측치 처리, 라벨 인코딩, 더미변수 변환(One-hot encoding), 정규화
- 데이터 분리 : 학습용, 검증용 데이터 분리
- 모형 훈련
- 모형 검증
- 모형 평가
- 모형 적용 : 실제 데이터로 예측을 해봄
- 결과 정리 : 보고서화, 시스템에 파이프라인 구축


In [ ]:
import pandas as pd

raw = pd.read_csv("./penguins.csv")

df = raw.copy()
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB
None


In [ ]:
df.describe()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
count,342.000000,342.000000,342.000000,342.000000
mean,43.921930,17.151170,200.915205,4201.754386
std,5.459584,1.974793,14.061714,801.954536
min,32.100000,13.100000,172.000000,2700.000000
25%,39.225000,15.600000,190.000000,3550.000000
50%,44.450000,17.300000,197.000000,4050.000000
75%,48.500000,18.700000,213.000000,4750.000000
max,59.600000,21.500000,231.000000,6300.000000


###################################
##   1-1. 데이터 전처리 - 결측치 처리
###################################

In [ ]:
#   독립변수(입력): 범주형, 연속형. 종속변수(출력): 범주형 --> 독립변수를 연속형으로 변환 후 앙상블 기법 활용.



# 데이터의 결측치를 확인
df.isna().sum()

,0
species,0
island,0
bill_length_mm,2
bill_depth_mm,2
flipper_length_mm,2
body_mass_g,2
sex,11


In [ ]:
# bill_length_mm 의 결측지를 중앙값으로 채움
df['bill_length_mm'] = df['bill_length_mm'].fillna(df['bill_length_mm'].median())

df.isna().sum()

,0
species,0
island,0
bill_length_mm,0
bill_depth_mm,2
flipper_length_mm,2
body_mass_g,2
sex,11


In [ ]:
df.dropna(inplace = True)

df.isna().sum()

,0
species,0
island,0
bill_length_mm,0
bill_depth_mm,0
flipper_length_mm,0
body_mass_g,0
sex,0


###################################
##   1-2. 데이터 전처리 - 라벨 인코딩
###################################

In [ ]:
###################################
#   1-2. 데이터 전처리 - 라벨 인코딩
###################################

print(df.head())

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

label = ['island', 'sex']

for col in label:
    df[col] = le.fit_transform(df[col])

df.head()

  species     island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
0  Adelie  Torgersen            39.1           18.7              181.0   
1  Adelie  Torgersen            39.5           17.4              186.0   
2  Adelie  Torgersen            40.3           18.0              195.0   
4  Adelie  Torgersen            36.7           19.3              193.0   
5  Adelie  Torgersen            39.3           20.6              190.0   

   body_mass_g     sex  
0       3750.0    Male  
1       3800.0  Female  
2       3250.0  Female  
4       3450.0  Female  
5       3650.0    Male  


,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,2,39.1,18.7,181.0,3750.0,1
1,Adelie,2,39.5,17.4,186.0,3800.0,0
2,Adelie,2,40.3,18.0,195.0,3250.0,0
4,Adelie,2,36.7,19.3,193.0,3450.0,0
5,Adelie,2,39.3,20.6,190.0,3650.0,1


In [ ]:
df['species'] = le.fit_transform(df['species'])
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,0,2,39.1,18.7,181.0,3750.0,1
1,0,2,39.5,17.4,186.0,3800.0,0
2,0,2,40.3,18.0,195.0,3250.0,0
4,0,2,36.7,19.3,193.0,3450.0,0
5,0,2,39.3,20.6,190.0,3650.0,1


###################################
##   1-3. 데이터 전처리 - 더미변수 변환
###################################

In [ ]:
###################################
#   1-3. 데이터 전처리 - 더미변수 변환
###################################

dummies = ['island', 'sex']

print(df.dtypes)

species                int64
island                 int64
bill_length_mm       float64
bill_depth_mm        float64
flipper_length_mm    float64
body_mass_g          float64
sex                    int64
dtype: object


In [ ]:
for col in dummies:
    df[col] = df[col].astype('category')

print(df.dtypes)

species                 int64
island               category
bill_length_mm        float64
bill_depth_mm         float64
flipper_length_mm     float64
body_mass_g           float64
sex                  category
dtype: object


In [ ]:
# One-hot encoding 을 진행해줌

df = pd.get_dummies(df, columns = dummies)
df.head()

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,island_0,island_1,island_2,sex_0,sex_1
0,0,39.1,18.7,181.0,3750.0,False,False,True,False,True
1,0,39.5,17.4,186.0,3800.0,False,False,True,True,False
2,0,40.3,18.0,195.0,3250.0,False,False,True,True,False
4,0,36.7,19.3,193.0,3450.0,False,False,True,True,False
5,0,39.3,20.6,190.0,3650.0,False,False,True,False,True


###################################
##   1-4. 데이터 전처리 - 정규화
###################################


In [ ]:
###################################
#   1-4. 데이터 전처리 - 정규화 (MinMaxScaler)
###################################

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

target = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']

df[target] = scaler.fit_transform(df[target])

df.head()

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,island_0,island_1,island_2,sex_0,sex_1
0,0,0.254545,0.666667,0.152542,0.291667,False,False,True,False,True
1,0,0.269091,0.511905,0.237288,0.305556,False,False,True,True,False
2,0,0.298182,0.583333,0.389831,0.152778,False,False,True,True,False
4,0,0.167273,0.738095,0.355932,0.208333,False,False,True,True,False
5,0,0.261818,0.892857,0.305085,0.263889,False,False,True,False,True


In [ ]:
df.describe()

# min, max가 모두 0, 1로 설정되어 있는 모습

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
count,333.000000,333.000000,333.000000,333.000000,333.000000
mean,0.918919,0.432465,0.483912,0.490966,0.418627
std,0.889718,0.198861,0.234433,0.237555,0.223671
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.269091,0.297619,0.305085,0.236111
50%,1.000000,0.450909,0.500000,0.423729,0.375000
75%,2.000000,0.600000,0.666667,0.694915,0.576389
max,2.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
# 필요 시 이상치 제거 IQR Method까지 진행

###################################
##   1-5. 데이터 전처리 - 데이터 분리
###################################

In [ ]:
df.head()

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,island_0,island_1,island_2,sex_0,sex_1
0,0,0.254545,0.666667,0.152542,0.291667,False,False,True,False,True
1,0,0.269091,0.511905,0.237288,0.305556,False,False,True,True,False
2,0,0.298182,0.583333,0.389831,0.152778,False,False,True,True,False
4,0,0.167273,0.738095,0.355932,0.208333,False,False,True,True,False
5,0,0.261818,0.892857,0.305085,0.263889,False,False,True,False,True


In [ ]:
###################################
#   1-5. 데이터 전처리 - 데이터 분리
###################################

from sklearn.model_selection import train_test_split

x_Train, x_Test, y_Train, y_Test = train_test_split(df.iloc[:, 1:], df.iloc[:,0], test_size = 0.2, random_state = 0)

print(x_Train.shape)
print(x_Test.shape)
print(y_Train.shape)
print(y_Test.shape)


(266, 9)
(67, 9)
(266,)
(67,)


###################################
##   2. 모형 훈련
###################################

In [ ]:
###################################
#   2. 모형 훈련
###################################

from sklearn.ensemble import RandomForestClassifier

model1 = RandomForestClassifier()

# 모델 학습
model1.fit(x_Train, y_Train)

# 모델 예측
pred1 = model1.predict(x_Test)

print(type(pred1))
print(pred1)

<class 'numpy.ndarray'>
[0 0 2 0 0 0 1 2 2 1 2 0 0 1 0 0 2 0 1 0 0 0 2 2 2 2 0 0 0 0 0 1 0 0 0 2 1
 0 1 0 2 2 0 0 0 0 0 0 2 0 0 0 2 2 0 0 0 0 0 2 0 1 0 2 0 0 2]


In [ ]:
from sklearn.ensemble import AdaBoostClassifier

model2 = AdaBoostClassifier()

# 모델 학습
model2.fit(x_Train, y_Train)

# 모델 예측
pred2 = model2.predict(x_Test)

print(type(pred2))
print(pred2)

<class 'numpy.ndarray'>
[0 0 2 0 0 0 0 2 2 0 2 0 2 0 0 0 2 0 0 0 0 0 2 2 2 2 0 0 0 0 0 0 0 0 0 2 0
 0 0 0 2 2 0 0 0 0 0 0 2 0 0 0 2 2 0 0 0 0 0 2 0 0 0 2 0 0 2]


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


###################################
##   3. 모형 성능 검증
###################################

In [ ]:
###################################
#   3. 모형 성능 검증
###################################

from sklearn.metrics import accuracy_score # 정답률을 알려주는 함수

acc1 = accuracy_score(y_Test, pred1)
acc2 = accuracy_score(y_Test, pred2)

print(f'acc1: {acc1}')
print(f'acc2: {acc2}')

acc1: 0.9701492537313433
acc2: 0.835820895522388


###################################
##   4. 하이퍼 파라미터 최적화
###################################

In [ ]:
###################################
#   4. 하이퍼 파라미터 최적화
###################################

#   ex.) number of perceptrons in neural net
#   ex.) number of estimators in randomforest
#   and so on

# 하이퍼 파라미터 튜닝 : estimator의 개수, layer의 개수, perceptron의 개수 => 모델의 구조에 영향을 미치는 것들을 조정하는 것

###################################
##   5. 결과 정리
###################################

In [ ]:
###################################
#   5. 결과 정리
###################################

result = le.inverse_transform(pred1)
print(result)



['Adelie' 'Adelie' 'Gentoo' 'Adelie' 'Adelie' 'Adelie' 'Chinstrap'
 'Gentoo' 'Gentoo' 'Chinstrap' 'Gentoo' 'Adelie' 'Adelie' 'Chinstrap'
 'Adelie' 'Adelie' 'Gentoo' 'Adelie' 'Chinstrap' 'Adelie' 'Adelie'
 'Adelie' 'Gentoo' 'Gentoo' 'Gentoo' 'Gentoo' 'Adelie' 'Adelie' 'Adelie'
 'Adelie' 'Adelie' 'Chinstrap' 'Adelie' 'Adelie' 'Adelie' 'Gentoo'
 'Chinstrap' 'Adelie' 'Chinstrap' 'Adelie' 'Gentoo' 'Gentoo' 'Adelie'
 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Gentoo' 'Adelie' 'Adelie'
 'Adelie' 'Gentoo' 'Gentoo' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie'
 'Gentoo' 'Adelie' 'Chinstrap' 'Adelie' 'Gentoo' 'Adelie' 'Adelie'
 'Gentoo']


# 실습 2. 2023 population annual

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

plt.rcParams['font.family'] ='NanumGothic'  # for displaying Hangul
# plt.rcParams['font.family'] ='AppleGothic'  # for displaying Hangul for Mac


##############################################################################
#
#   0. 데이터 읽어오기
#
##############################################################################

encoding = 'EUC-KR'

df = pd.read_csv('./2023_population_annual.csv', encoding = encoding)
df.head()

,행정구역,2023년_계_총인구수,2023년_계_연령구간인구수,2023년_계_0세,2023년_계_1세,2023년_계_2세,2023년_계_3세,2023년_계_4세,2023년_계_5세,2023년_계_6세,...,2023년_계_91세,2023년_계_92세,2023년_계_93세,2023년_계_94세,2023년_계_95세,2023년_계_96세,2023년_계_97세,2023년_계_98세,2023년_계_99세,2023년_계_100세 이상
0,서울특별시 (1100000000),"9,386,034","9,386,034","37,771","41,087","43,013","43,637","47,506","50,291","55,100",...,"9,131","6,569","5,341","4,468","3,233","2,360","1,387",887,670,"1,448"
1,서울특별시 종로구 (1111000000),"139,417","139,417",390,460,479,470,543,552,655,...,201,138,110,87,71,56,38,17,19,35
2,서울특별시 종로구 청운효자동(1111051500),"11,349","11,349",40,44,44,48,51,50,68,...,15,15,7,5,7,4,2,1,3,3
3,서울특별시 종로구 사직동(1111053000),"8,998","8,998",24,27,45,32,46,47,51,...,15,12,10,11,9,5,2,3,3,5
4,서울특별시 종로구 삼청동(1111054000),"2,248","2,248",6,5,7,5,5,11,14,...,9,3,4,3,2,0,2,0,0,2


In [ ]:
##############################################################################
#
#   1. 데이터 전처리(pre-processing)
#
##############################################################################

#   원본 데이터에서 제목, 총합, 평균 등 필요없는 데이터를 빼고 분석 대상이 될 데이터만 가져옵니다.
#   만약 값이 입력되어 있지 않은 셀 등이 있으면(결측치, missing values), 평균, 중간값 등으로 채워주어야 하는 경우도 있습니다.


#   결측값 처리

def convertStrToInt(strInt):
    if not isinstance(strInt, int):
        result = int(strInt.replace(',', ''))
    else:
        result = strInt

    return result


#   라벨 인코딩

#   더미변수 처리

#   데이터 분리




In [ ]:
##############################################################################
#
#   2. 모형 개발, 적용, 평가
#
##############################################################################

#   모형 개발
def calcRMSE(tar, ref): # calculate RMSE(Root Mean Square Error)

    RMSE = None   # initial value

    accSquareError = 0.0   # accumulated square error with initial value 0

    if len(tar) != len(ref):
        pass
    else:
        if len(tar) == 0 or len(ref) == 0:
            pass
        else:
            for idx in range(len(ref)):
                error = tar[idx] - ref[idx]
                squareError = error*error
                accSquareError = accSquareError + squareError
            meanSquareError = accSquareError/len(ref)
            RMSE = np.sqrt(meanSquareError)

    return RMSE

#   모형 훈련(모형 적용)

#   나의 동네 인구구성 찾고 읽기
#   전처리를 마친 데이터에서 행정 구역명(예) myHomeTown = '남구 효곡동')을 기준으로 내가 살고 있는 동네 인구구성이 들어있는 행을 찾습니다.
#   찾은 행에서 행정 구역명을 제외한 인구구성 데이터만 가져와 변수에 저장합니다.
#   인구구성 데이터만 가지고 있어야 다른 동네 인구구성 데이터와 비교할 수 있기 때문입니다.


#   모형 적용 - 인구구성 데이터 비교하기
#   위 '나의 동네 인구구성 찾고 읽기' 과정에서 얻은 내가 사는 동네 인구구성 데이터와 다른 동네들의 인구구성을 비교해줍니다.
#   인구 구성이 얼마나 비슷한지 최소자승오차(RMSE) 개념을 이용하여 그 오차를 수치로 측정해줍니다.
#   이 최소자승오차가 가장 작은 동네를 찾으면 이 동네가 우리 동네와 인구구성이 가장 비슷한 동네입니다. 찾은 동네를 변수에 저장해줍니다.



In [ ]:
##############################################################################
#
#   3. 결과 정리
#
##############################################################################

#   인구구성 데이터 시각화
#   위 과정에서 찾은 '내가 살고 있는 동네'와 '인구구성이 가장 비슷한 동네'의 인구구성을 그래프로 나타내줍니다.
#   시각화를 통해 그 결과가 얼마나 유사한지 직접 확인해 볼 수 있습니다.